In [37]:
require 'nn';


In [63]:
require 'nn'
--[[
This file contains the code definition of our complex example network in raw Torch
using nn.Table modules.  Since nngraph is not used, this network is defined in
reverse order in terms of sequential segments and branches.
]]--


-- the final layer of our base network 
net3 = nn.Sequential()
net3:add(nn.SpatialConvolution(128,256,3,3,1,1,1,1))
net3:add(nn.View(-1))

-- split the last branch 
branch2b = nn.ParallelTable()
branch2b:add(nn.SpatialConvolution(128,256,3,3,1,1,1,1))
branch2b:add(nn.SpatialConvolution(128,256,3,3,1,1,1,1))

-- here we add max pooling and use nn.Replicate() --> nn.SplitTable
-- to allow for a ParallelTable to split the branch 
branch2a = nn.Sequential()
branch2a:add(nn.SpatialMaxPooling(2,2,2,2))
branch2a:add(nn.Replicate(4))
branch2a:add(nn.SplitTable(1))
branch2a:add(branch2b)

-- second (final) branch from base of network
branch2 = nn.ParallelTable()
branch2:add(branch2a)
branch2:add(net3)

-- using nn.Sequential() for part 2 of base network 
net2 = nn.Sequential() 
net2:add(nn.SpatialConvolution(64,128,3,3,1,1,1,1))
net2:add(nn.ReLU())
net2:add(nn.SpatialConvolution(128,128,3,3,1,1,1,1))
net2:add(nn.ReLU())
net2:add(nn.Replicate(4))
net2:add(nn.SplitTable(1))
net2:add(branch2)

-- split from first branch 
branch1b = nn.ParallelTable()
branch1b:add(nn.SpatialConvolution(64,64,3,3,1,1,1,1))
branch1b:add(nn.SpatialConvolution(64,64,3,3,1,1,1,1))

-- add a max pooling layer to branch and then prepare to split 
branch1a = nn.Sequential()
branch1a:add(nn.SpatialMaxPooling(2,2,2,2))
branch1a:add(nn.Replicate(4))
branch1a:add(nn.SplitTable(1))
branch1a:add(branch1b)

-- first branch from the base of the network
branch1 = nn.ParallelTable()
branch1:add(branch1a)
branch1:add(net2)

-- the first layers of the network 
net = nn.Sequential()
net:add(nn.SpatialConvolution(3,64,3,3,1,1,1,1))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64,64,3,3,1,1,1,1))
net:add(nn.ReLU())
net:add(nn.Replicate(4))
net:add(nn.SplitTable(1))
net:add(branch1)


In [65]:
require 'nn'
require 'nngraph'
--[[
This file contains the code definition of our complex example network using nngraph
in combination with some of the nn.Table modules that we defined.
]]--

input = nn.Identity()() -- 

L1 = nn.SpatialConvolution(3,64,3,3,1,1,1,1)(input)
r1 = nn.ReLU()(L1)
L2 = nn.SpatialConvolution(64,64,3,3,1,1,1,1)(r1)
r2 = nn.ReLU()(L2)
L3 = nn.SpatialConvolution(64,128,3,3,1,1,1,1)(r2)
r3 = nn.ReLU()(L3)
L4 = nn.SpatialConvolution(128,128,3,3,1,1,1,1)(r3)
r4 = nn.ReLU()(L4)
r5 = nn.View(-1)(nn.SpatialConvolution(128,256,3,3,1,1,1,1)(r4))

b2 = nn.SpatialMaxPooling(2,2,2,2)(r4)
b2a = nn.SpatialConvolution(128,256,3,3,1,1,1,1) 
b2b = nn.SpatialConvolution(128,256,3,3,1,1,1,1)

leaf2a = nn.View(-1)(b2a(b2))
leaf2b = nn.View(-1)(b2b(b2))

leaf2 = nn.JoinTable(1)({leaf2a,leaf2b})

b1 = nn.SpatialMaxPooling(2,2,2,2)(r2)
b1a = nn.SpatialConvolution(64,64,3,3,1,1,1,1)
b1b = nn.SpatialConvolution(64,64,3,3,1,1,1,1)

leaf1a = nn.View(-1)(b1a(b1))
leaf1b = nn.View(-1)(b1b(b1))

leaf1 = nn.JoinTable(1)({leaf1a,leaf1b})

output = nn.JoinTable(1)({leaf1,r5,leaf2})

net = nn.gModule({input},{output})
out = net:forward(torch.Tensor(1,3,10,10))
graph.dot(net.fg, 'Forward Graph', 'complexgraph')
graph.dot(net.bg, 'Backward Graph','bcomplexgraph')

In [66]:
out:size()

 41600
[torch.LongStorage of size 1]



In [58]:
out = net:forward(torch.Tensor(1,3,10,10))

In [59]:
out:size()

 25600
[torch.LongStorage of size 1]



In [44]:
tip1:size()

 40000
[torch.LongStorage of size 1]



In [45]:
tip2:size()

 40000
[torch.LongStorage of size 1]



In [ ]:
input = torch.Tensor(1,3,50,50)

In [40]:
out = net:forward(input)

In [41]:
net

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
  (1): nn.SpatialConvolution(3 -> 64, 3x3, 1,1, 1,1)
  (2): nn.ReLU
  (3): nn.SpatialConvolution(64 -> 64, 3x3, 1,1, 1,1)
  (4): nn.Replicate
  (5): nn.SplitTable
}
{
  gradInput : DoubleTensor - empty
  modules : 
    {
      1 : 
        nn.SpatialConvolution(3 -> 64, 3x3, 1,1, 1,1)
        {
          padW : 1
          nInputPlane : 3
          output : DoubleTensor - size: 1x64x50x50
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor


          dH : 1
          dW : 1
          nOutputPlane : 64
          padH : 1
          kH : 3
          finput : DoubleTensor - size: 1x27x2500
          weight : DoubleTensor - size: 64x3x3x3
          gradWeight : DoubleTensor - size: 64x3x3x3
          fgradInput : DoubleTensor - empty
          kW : 3
          bias : DoubleTensor - size: 64
          gradBias : DoubleTensor - size: 64
        }
      2 : 
        nn.ReLU
        {
          inplace : false
          threshold : 0
          _type : torch.DoubleTensor
          output : DoubleTensor - size: 1x64x50x50
          gradInput : DoubleTensor - empty
          val : 0
        }
      3 : 
        nn.SpatialConvolution(64 -> 64, 3x3, 1,1, 1,1)
        {
          padW : 1
          nInputPlane : 64
          output : DoubleTensor - size: 1x64x50x50
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
          dH : 1
          dW : 1
          nOutputPlane : 64
          padH : 1
          kH 